In [1]:
import pandas as pd
import numpy  as np

from scipy   import sparse
from sklearn.feature_extraction.text import TfidfVectorizer

from adarank import AdaRank
from metrics import NDCGScorer

In [14]:
df = pd.read_excel("base_data.xlsx")
df

,Query id,loinc_num,f0,f1,f2,f3,Y
0,1,1003-3,Indirect antiglobulin test complement specific...,Indirect antiglobulin test complement specific...,Ser / Plas,ACnc,30
1,1,10331-7,Rh Type in Blood,Rh,Bld,Type,17
2,1,1250-0,Major crossmatch interpretation,Major crossmatch,Ser / Plas,Imp,33
3,1,13317-3,Methicillin resistant Staphylococcus aureus Pr...,Staphylococcus aureus methicillin resistant is...,XXX,ACnc,56
4,1,14423-8,Bilirubin total Mass / volume in Synovial fluid,Bilirubin,Synv fld,MCnc,58
...,...,...,...,...,...,...,...
196,3,883-9,ABO group Type in Blood,ABO group,Bld,Type,44
197,3,890-4,Blood group antibody screen Presence in Serum ...,Blood group antibody screen,Ser / Plas,ACnc,6
198,3,925-8,Blood product disposition Type,Blood product disposition,BPU,Type,9
199,3,933-2,Blood product type,Blood product type,BPU,Type,27


In [2]:
df = pd.read_excel("data copy.xlsx", header=0)
#df['id_doc'] =  df["id_doc"].apply(lambda x : [x])

In [3]:
#tfidf = TfidfVectorizer().fit(df["features"])
#df["features"] = df["features"].apply(lambda x : list(tfidf.transform([x]).data))

In [ ]:
def tokenize(text):
    text = re.split('\s+' ,text)
    return [x.lower() for x in text]

data['tokens'] = data['removed_punc'].apply(lambda msg : tokenize(msg))
data.head()

from nltk.tokenize import word_tokenize

data['tokens'] = data['removed_punc'].apply(lambda msg : word_tokenize(msg.lower()))
data.head()

def return_sentences(tokens):
    return " ".join([word for word in tokens])

data['clean_text'] = data['lemma_words'].apply(lambda x : return_sentences(x))
data.head()

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_vect = tfidf.fit_transform(data['clean_text'])
tfidf_vect.shape

tfidf_vect_2 = tfidf.transform(['hello world jokwdwqdqwing'])
print(tfidf_vect_2)

def get_doc_score(doc, query_word_ids):
    doc_score = 0
    for word_id in query_word_ids:
        doc_score += doc[(0,word_id)]
    return doc_score


query = "partying is great"
query = tfidf.transform([query])
query_word_ids = query.indices

documents_scores= [[doc_id,get_doc_score(tfidf_vect[doc_id], query_word_ids)] for doc_id in range(0,5572)]
documents_scores.sort(key=lambda x:x[1], reverse=True)

top_10 = documents_scores[:10]
top_10 = map(lambda x: x[0], top_10)

data.iloc[top_10] 

def get_l_norm(doc, query_word_ids):
    doc_l_norm = 0
    
    for word_id in query_word_ids:
        word_tfidf  = doc[(0,word_id)]
        doc_l_norm += pow(word_tfidf,2)
        
    doc_l_norm = np.sqrt(doc_l_norm)
    return doc_l_norm

def normalize_vector(doc, query_word_ids):
    l_norm = get_l_norm(doc, query_word_ids)

    return [doc[(0,word_id)] / l_norm if l_norm else 0 for word_id in query_word_ids]

def get_doc_cos_score(doc, query):
    return sum([doc[i] * query[i] for i in range(len(doc))]) 
    

query = "partying is great"
query = tfidf.transform([query])

query_word_ids = query.indices
norm_query     = normalize_vector(query, query_word_ids)

documents_scores = [[doc_id,get_doc_cos_score(normalize_vector(tfidf_vect[doc_id], query_word_ids), norm_query)] for doc_id in range(0,5572)]
documents_scores.sort(key=lambda x:x[1], reverse=True)

top_10 = documents_scores[:10]
top_10 = map(lambda x: x[0], top_10)

data.iloc[top_10] 

In [5]:
qid = df["Qid"].to_numpy()
X   = df["features"].to_list()
Y   = df["rank"].to_numpy()

In [6]:
max_len = max(df["features"].apply(lambda x: len(x)))

X_t = []
for i in range(len(X)):
    if len(X_t) == 0:
        X_t = np.array([X[i] + [0]*(max_len-len(X[i]))])
    else:
        X_t = np.append(X_t, [X[i] + [0]*(max_len-len(X[i]))], axis=0)
        
X = sparse.csr_matrix(X_t)

In [8]:
model = AdaRank(max_iter=1, scorer=NDCGScorer(), verbose=True)
model.fit(X[:int(X.shape[0]*0.75)], Y[:int(len(Y)*0.75)] , qid[:int(len(qid)*0.75)], X[int(X.shape[0]*0.75):int(X.shape[0]*0.85)], Y[int(len(Y)*0.75):int(len(Y)*0.85)] , qid[int(len(qid)*0.75):int(len(qid)*0.85)])

[[0.411183   0.36443254 0.07372316 ... 0.411183   0.15635892 0.        ]
 [0.57949667 0.75151617 0.1095066  ... 0.         0.         0.        ]
 [0.11002139 0.11002139 0.62475294 ... 0.         0.         0.        ]
 ...
 [0.16603081 0.19693724 0.1934318  ... 0.         0.         0.        ]
 [0.11824172 0.14025227 0.13775581 ... 0.         0.         0.        ]
 [0.28687972 0.28687972 0.27766001 ... 0.         0.         0.        ]]
[0.411183   0.57949667 0.11002139 0.16918139 0.1184487  0.26642967
 0.12205901 0.16881221 0.06581981 0.11800321 0.09974453 0.16011424
 0.11801507 0.16603081 0.11824172 0.28687972 0.20463556 0.26111996
 0.26111996 0.26111996 0.26111996 0.66661139 0.89429371 0.12864699
 0.09974453 0.1328108  0.10251911 0.13219479 0.15962797 0.11391648
 0.1196157  0.1181694  0.10628751 0.09939322 0.2260641  0.10733624
 0.26111996 0.1453909  0.14281587 0.10118161 0.14422971 0.08557295
 0.13712794 0.13192395 0.31526485 0.14469995 0.14469995 0.30179356
 0.13228351 0.081627

AssertionError: 

In [ ]:
pred= model.predict(X, qid)

In [ ]:
for i in range(len(pred)):
    print(i, pred[i])